# Explore Graph using Neo4j

In this section we will explore the graph we just created using Cypher and the Graph Data Science (GDS) library for Neo4j.

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import py2neo
import umap

from py2neo import Graph

%matplotlib inline

In [ ]:
DATA_DIR = "../../../data/project-1"

NEO4J_DUMP = os.path.join(DATA_DIR, "neo4j-dump.json")
NEO4J_LABELS = os.path.join(DATA_DIR, "neo4j-labels.tsv")

NODE_VEC_FILE = os.path.join(DATA_DIR, "stat-abstract-vectors.tsv")

## Connect to Neo4j server

The code below connects to the Neo4j server using the `bolt` interface.

For a sanity check, enter a Cypher query to count the number of nodes of type `Article` in the graph. You should see 50427 nodes

In [ ]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "admin"))

In [ ]:
# samity check
result = graph.run(
    ## your code here
    """
    
    """
    ## end your code here
)
result

## Create subgraph for GDS algorithms

GDS needs a subgraph to run, even if we are running the algorithm on the entire graph. So we create a virtual subgraph as shown below, with the `Article` node type and `SIMILAR_TO` relationship type.

In [ ]:
result = graph.run("""
CALL gds.graph.create('av-graph-gds', 'Article', 'SIMILAR_TO')
""")
result

## Important Nodes

### Degree Centrality

A measure of importance of a node in a graph is the number of neighbors it is connected to. Look up the [documentation for the GDS Degree Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/) to learn more about this measure and provide the Cypher query in the code block below to generate the degree centrality for nodes in the subgraph, and return the top 10 nodes with the highest degree centrality. Include the `doc_id`, `title`, `category`, and the centrality `score` in your output.

In [ ]:
def show_result_as_dataframe(result, colnames, filterset=None):
    result_lod = []
    already_seen_docids = set()
    for row in result:
        doc_id, title, category, page_rank = row
        if filterset is not None and doc_id in filterset:
            continue
        if doc_id in already_seen_docids:
            continue
        result_dict = {}
        for i, colname in enumerate(colnames):
            result_dict[colname] = row[i]
        result_lod.append(result_dict)
        already_seen_docids.add(doc_id)
    result_df = pd.DataFrame(result_lod)
    return result_df.head(10)

In [ ]:
result = graph.run(
    # your code here
    """

    """
    # end your code here
)
show_result_as_dataframe(result, ["doc_id", "title", "category", "score"])

### PageRank

Another useful measure of central tendency (and hence importance) is the PageRank algorithm. It is more involved than Degree Centrality, and takes into account not only the number of neighbors a node has, but also how important they are. The importance of neighbor nodes is, in turn, dependent on the importance of their neighbors. Read about PageRank in the [GDS Documentation page for PageRank](https://neo4j.com/docs/graph-data-science/current/algorithms/page-rank/), then fill out the Cypher code snippet to compute the PageRank and return the top 10 articles by Page Rank (higher is better). 

Set the configuration options as `{ maxIterations: 20, dampingFactor: 0.85 }`.

In [ ]:
result = graph.run(
    # your code here
    """
    
    """
    # end your code here
)
show_result_as_dataframe(result, ["doc_id", "title", "category", "score"])

### Between-ness Centrality

A slightly different measure of centrality is Between-ness Centrality. It identifies nodes that are placed as bridges between two relatively dense clusters. In practice they represent articles that bridge or cross thematic boundaries. They are expensive to compute for dense graphs since they involve computing paths between all pairs of nodes. In order to reduce the computation, we will configure the call to the between-ness centrality algorithm to only look at around 1000 nodes. Check out the [GDS Documentation for Betweenness Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/) and fill in the Cypher query to find the nodes with the top 10 highest values for between-ness centrality.

Configure a `samplingSize` of 1000 nodes.

**NOTE: Expect this call to take some time to complete, even with `samplingSize` set to 1000.**

In [ ]:
result = graph.run(
    # your code here
    """

    """
    ## end of your code here
)
show_result_as_dataframe(result, ["doc_id", "title", "category", "score"])

## Community Detection

### Louvain Modularity

Louvain is a popular community detection algorithm. It works by maximizing the modularity of the created cluster, where the modularity quantifies the quality of assignment of nodes to communities compared to a random graph.

For our community detection algorithms, we will adopt a two step procedure. First we will call the algorithm to write the generated community as a node property, then we will count the number of nodes in each community.

To guide the community creation process, we will use the `category` as a seed property. Because Louvain algorithm needs an integer seed category, we will be writing out a derived field `cat_id` as shown below. 

Once we have our derived property `cat_id` we can use it as the seed property for the GDS Louvain algorithm.

In [ ]:
cat2id = {
    "stat.AP": 0,
    "stat.CO": 1,
    "stat.ME": 2,
    "stat.ML": 3,
    "stat.OT": 4, 
    "stat.TH": 5
}
for category, cat_id in cat2id.items():
    cypher_query = """MATCH (n {category: "%s"}) SET n.cat_id = %d""" % (category, cat_id)
    print(cypher_query)
    graph.run(cypher_query)

In [ ]:
result = graph.run("""
CALL gds.graph.create('av-graph-gds-1','Article','SIMILAR_TO', 
    {
        nodeProperties: { catId: 'cat_id' }
    })
""")
result

Now read about the [Louvain Community Detection Algorithm in GDS](https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/), and complete the code below to write out the commmunity predicted by the Louvain algorithm into the node property `community_lv`.

Remember to set the `seedProperty` in the configuration as discussed above!

In [ ]:
result = graph.run(
    # your code here
    """

    """
    # end of your code here
)
result

In [ ]:
num_communities = result.data()[0]["communityCount"]
num_communities

One way to find the number of nodes in each community is through the Cypher query:

```
MATCH (a:Article)
RETURN a.community_lp, COUNT(a) AS num_articles
```

But this returns only the top 3 counts. The code below loops through the different community IDs and for each value of `community_lv` it will report the counts.

In [ ]:
def article_count_by_community(graph, comm_field_name, num_communities):
    community_counts = []
    for i in range(num_communities):
        result_row = graph.run(
            """
                MATCH (a:Article {%s: %d})
                RETURN a.%s, COUNT(a) AS num_articles
            """ % (comm_field_name, i, comm_field_name)
        )
        num_articles = result_row.data()[0]["num_articles"]
        community_counts.append((i, num_articles))
    return community_counts

In [ ]:
print(article_count_by_community(graph, "community_lv", num_communities))

### Label Propagation

Label Propagation is another community detection algorithm that works by propagating labels across the network. You can read about [Label Propagation in the GDS Documentation](https://neo4j.com/docs/graph-data-science/current/algorithms/label-propagation/).

We will follow a similar strategy to run the Label Propagation algorithm on our graph as we did with the Louvain algorithm. Here we will write the community ID predicted by the Label Propagation algorithm into the `community_lp` node property. As with Louvain, we will see our Label Propagation algorithm with the `category` as the `seedProperty`.

Once the algorithm has finished running, we will look at the counts of nodes in each community.

Complete the code blocks below.

In [ ]:
result = graph.run(
    ## your code here
    """

    """
    ## end your code here
)
result

In [ ]:
num_communities = result.data()[0]["communityCount"]
num_communities

In [ ]:
print(article_count_by_community(graph, "community_lp", num_communities))

### Visualizing Communities

We will visualize the communities that was implicitly known via the `category` property against the communities predicted by the Louvain and Label Propagation algorithms respectively.

To do that, we will need to export the data from Numpy and convert the data into Numpy matrices. To export the data, we can run the following Cypher query.

Note that you will also need to install UMAP by running the following command:

```
pip install umap-learn
```

In [ ]:
graph.run("""CALL apoc.export.json.all("/tmp/neo4j-dump.json")""")

The dump contains all nodes and relationships in JSON-L format, one line per node and one line per relationship. We reformat this to extract just a TSV file with `doc_id` of the article, the implicit `category` label, and the predicted `community_lv` and `community_lp` values from Louvain and Label Propagation algorithms respectively.

In [ ]:
num_nodes = 0
flab = open(NEO4J_LABELS, "w")
with open("/tmp/neo4j-dump.json", "r") as fdump:
    for line in fdump:
        data = json.loads(line.strip())
        if data["type"] == "node" and data["labels"][0] == "Article":
            doc_id = data["properties"]["doc_id"]
            category = data["properties"]["category"]
            community_lv = data["properties"]["community"]
            community_lp = data["properties"]["community_lp"]
            flab.write("{:s}\t{:s}\t{:d}\t{:d}\n".format(doc_id, category, community_lv, community_lp))
            if num_nodes % 10000 == 0:
                print("{:d} node labels written".format(num_nodes))
            num_nodes += 1

print("{:d} node labels written, COMPLETE".format(num_nodes))
flab.close()

### Extract labels

In [ ]:
y_cat, y_lv, y_lp = [], [], []
num_read = 0
with open(NEO4J_LABELS, "r") as flab:
    for line in flab:
        if num_read % 10000 == 0:
            print("{:d} labels read".format(num_read))
        docid, label_c, label_lv, label_lp = line.strip().split('\t')
        y_cat.append(label_c)
        y_lv.append(int(label_lv))
        y_lp.append(int(label_lp))
        num_read += 1
        
print("{:d} labels read, COMPLETE".format(num_read))
len(y_cat), len(y_lv), len(y_lp)

In [ ]:
unique_cats = [cat for cat in list(set(y_cat))]
cid_to_cat = {i:c for i, c in enumerate(unique_cats)}
cat_to_cid = {v:k for k, v in cid_to_cat.items()}
cat_to_cid

In [ ]:
y_c = np.array([cat_to_cid[y] for y in y_cat])
y_c.shape

### Extract Node vectors and project to 2-D

Read the [UMAP Documentation page](https://umap-learn.readthedocs.io/en/latest/) and complete the code to project the matrix X of node vectors from 300 dimensions down to 2 using UMAP.

In [ ]:
docid_to_idx = {}
vecs = []
num_read = 0
with open(NODE_VEC_FILE, "r") as fvec:
    for i, line in enumerate(fvec):
        if num_read % 10000 == 0:
            print("{:d} articles read".format(num_read))
        docid, vec_str = line.strip().split('\t')
        vec = [float(x) for x in vec_str.split(',')]
        vecs.append(vec)
        docid_to_idx[docid] = i
        num_read += 1

print("{:d} articles read, COMPLETE".format(num_read))

In [ ]:
X = np.array(vecs)
X.shape

In [ ]:
umap_r = umap.UMAP()

## your code here


## end your code here

X_u.shape

### Visualize clustering using category labels

In [ ]:
plt.scatter(X_u[:, 0], X_u[:, 1], c=y_c.astype(np.int32), cmap=plt.cm.viridis)
plt.show()

### Visualize clustering using Louvain community predictions

Complete the code below to visualize clusters using the `community_lv` predictions by the Louvain algorithm. It is similar to the code we showed above. remember that the `color (c)` parameter takes a numpy array of `dtype=np.int32`.

In [ ]:
## your code here

## end your code here

### Visualize clusters using Label Propagation Community predictions

Now complete the code to visualize clusters using the `community_lp` predictions made by the Label Propagation algorithm. It is similar to the code you completed for the `community_lv` predictions from the Louvain algorithm.

In [ ]:
## your code here

## end of your code here

Clearly, of the three, the best clustering is produced by the Louvain algorithm, followed by Label Propagation.

## Recommending similar articles

We seek to answer the question: given some node in a graph, what are some similar articles? This can be found using the Personalized PageRank, a variant of the [PageRank Algorithm](https://neo4j.com/docs/graph-data-science/current/algorithms/page-rank/).

The difference between PageRank and Personalized PageRank is that the random surfer returns to a node or group of node instead of jumping to a random point in the web with some probability.

Let us try to find similar articles to `0812.1124` the top article by degree centrality that we found earlier. Write a Cypher query to get 5 of its neighbors. You want to get the internal node ID of the neighbors, not the `doc_id`. You can use the `ID()` function to extract the internal node ID for a matched node.

In [ ]:
result = graph.run(
    ## your code here
    """

    """
    ## end your code here
)
neighbors = list([r["doc_id"] for r in result])
neighbors

The `doc_id`s you identified plus the original node's `doc_id` will form our "neighborhood" for personalized PageRank computations.

In [ ]:
neighbors.append('0812.1124')
neighbors

In [ ]:
result = graph.run(
    """
    MATCH (a:Article) WHERE a.doc_id in [%s]
    CALL gds.pageRank.stream('av-graph-gds', 
        {
          maxIterations: 20,
          dampingFactor: 0.85,
          sourceNodes: [a]
        })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).doc_id, gds.util.asNode(nodeId).title,
           gds.util.asNode(nodeId).category, score
    ORDER BY score DESC
    LIMIT 30
    """ % (",".join(["'{:s}'".format(nbr) for nbr in neighbors]))
)
show_result_as_dataframe(result, ["doc_id", "title", "category", "score"], filterset=set(neighbors))

## Clean up

In [ ]:
graph.run("""CALL gds.graph.drop('av-graph-gds')""")

In [ ]:
graph.run("""CALL gds.graph.drop('av-graph-gds-1')""")